<a href="https://colab.research.google.com/github/2019zhou/flask-Couplet/blob/main/couplet1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
wget -O "fairseq.tar.gz" https://lxylab.oss-cn-shanghai.aliyuncs.com/fairseq/fairseq.tar.gz?Expires=1613892173&OSSAccessKeyId=TMP.3KefFe833Phg9GYBdLccdwPsLocBjR6YnV9tbJMjYdo2Av1is9hgpgm9AvGeZufQXKvmaoMKEPPBBMBxTuQkcktbNQi2Cg&Signature=xRmS07NFI9XhPF9nc9AqxzW00oA%3D&versionId=CAEQGRiBgIDosIKNvhciIDZkZTM0M2EwZjViYjRjNjNhZTYxZjIwN2QxNjc3NDk4&response-content-type=application%2Foctet-stream
rm -rf ./fairseq

In [ ]:
%%bash
tar -zxvf "fairseq.tar.gz"
cd fairseq
pip install -e ./
cd ..

In [ ]:
%%bash
mkdir -p /content/fairseq/checkpoints/fconv

In [ ]:
%%bash
git clone https://github.com/luxinyu1/Chinese-couplet.git

Cloning into 'Chinese-couplet'...


In [ ]:
%%bash
wget https://lxylab.oss-cn-shanghai.aliyuncs.com/chinese-couplet/checkpoint_best.pt?Expires=1613749934&OSSAccessKeyId=TMP.3Kf8DcbtTrmbyK1db8w9NnGyN4Wt7c3SEkAGL3v2UDzkc2CRusmiWLEjdSBVPatN4N7mEDcxBHipFrSxkfvKRA11ptXbYn&Signature=rNsVORC2gK5ioKszqsWN7kcQ0uU%3D&versionId=CAEQGRiBgMCl1vXfvRciIGJhNTkyYTU0NzdhNTRmNDA5ZjY3ZjIzNWI2NDQzNTNi&response-content-type=application%2Foctet-stream

In [5]:
%%bash
mv checkpoint_best.pt?Expires=1613749934 checkpoint_best.pt

mv: cannot stat 'checkpoint_best.pt?Expires=1613749934': No such file or directory


In [ ]:
%%bash
cp /content/drive/MyDrive/couplets/checkpoint_best.pt -r /content/

In [7]:
!cp checkpoint_best.pt -r /content/fairseq/checkpoints/fconv/

In [ ]:
%cd fairseq

/content/fairseq


In [ ]:
%cd /content/

/content


In [21]:
%%bash
fairseq-interactive /content/Chinese-couplet/couplet-bin \
  --path /content/checkpoint_best.pt --source-lang "src" \
  --target-lang "dst" --beam 5 --input /content/input.txt > /content/output.txt

In [11]:
%%bash
grep ^H /content/output.txt | cut -f3- > /content/output2.txt

In [15]:
!pip install Flask

In [16]:
!pip install flask-ngrok

In [18]:
!git clone https://github.com/2019zhou/flask-Couplet.git

Cloning into 'flask-Couplet'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [19]:
%%bash
cd flask-Couplet
tar -zxvf "templates.tar.gz"
mv templates /content/templates

templates/
templates/index.html
templates/predict.html
templates/.ipynb_checkpoints/


In [12]:
import subprocess
import re 

def getresult(firstLine):
  print(firstLine)
  firstLine = firstLine.strip()
  strtmp = ""
  for x in firstLine:
    strtmp += x + " "
  filename = "/content/input.txt"
  file = open(filename, "w")
  file.write(strtmp)
  file.close()
  p = subprocess.Popen(['fairseq-interactive /content/fairseq/couplet-bin \
  --path /content/fairseq/checkpoints/fconv/checkpoint_best.pt --source-lang "src" \
  --target-lang "dst" --beam 5 --input %s > /content/output.txt'%(filename)],shell=True, stdout=subprocess.PIPE)  
  out, err = p.communicate()  
  p2 = subprocess.Popen(['grep ^H /content/output.txt | cut -f3- > /content/output2.txt'],shell=True)
  out2, err2 = p2.communicate()
  f = open("/content/output2.txt")
  res = f.read()
  f.close()
  return "".join(res.split())


In [51]:
import subprocess
import re 

def getresult(firstLine):
  firstLine = firstLine.strip()
  strtmp = ""
  for x in firstLine:
    strtmp += x + " "
  p = subprocess.Popen(['echo %s | fairseq-interactive /content/fairseq/couplet-bin \
  --path /content/fairseq/checkpoints/fconv/checkpoint_best.pt --source-lang "src" \
  --target-lang "dst" --beam 5 | grep ^H | cut -f3-'%(strtmp)],shell=True, stdout=subprocess.PIPE)  
  res, err = p.communicate()
  res = res.decode('utf-8').splitlines()
  return "".join(res[0].split())

In [48]:
getresult("春眠不知晓")

春眠不知晓


'秋雨不知音'

In [ ]:
%cd /content

/content


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

# def getresult(firstLine):
#     p = subprocess.Popen('fairseq-interactive couplet-bin --source-lang "src" --target-lang "dst" \
#     --path checkpoint_best.pt \
#     --beam 5 ',shell=True,stdout=subprocess.PIPE)  
#     out,err = p.communicate()  
#     line1 = ""
#     print(out)
#     for line in out.splitlines():
#         print(line)
#         line1 = line
#     return line1


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'GET':
        return render_template('index.html')
    elif request.method == 'POST':
        secondlinetmp = getresult(request.form['firstLine'])
        Couplets = {
            'firstLine':request.form['firstLine'],
            'secondLine': secondlinetmp,
        }
        return render_template('predict.html', **Couplets)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://83d57489ab73.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


## 返回json文件
- 做成API样式

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

# def getresult(firstLine):
#     p = subprocess.Popen('fairseq-interactive couplet-bin --source-lang "src" --target-lang "dst" \
#     --path checkpoint_best.pt \
#     --beam 5 ',shell=True,stdout=subprocess.PIPE)  
#     out,err = p.communicate()  
#     line1 = ""
#     print(out)
#     for line in out.splitlines():
#         print(line)
#         line1 = line
#     return line1


@app.route('/', methods=['GET'])
def index():
    if request.method == 'GET':
      secondlinetmp = getresult(request.form['firstLine'])
      if secondlinetmp:
        Couplets = {
            "code": 200,
            "status": "success",
            "data": [{
                'firstLine':request.form['firstLine'],
                'secondLine': secondlinetmp
            }]
        }
        return Couplets
      else:
        return render_template('index.html')    

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://18cd54d79fee.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/', methods=['GET'])
def index():
    if request.method == 'GET':
        secondlinetmp = getresult(request.args.get('firstLine'))
        if secondlinetmp:
            Couplets = {
                "code": 200,
                "status": "success",
                "data": [{
                    'firstLine':request.args.get('firstLine'),
                    'secondLine':secondlinetmp
                }]
            }
            print(Couplets)
            return Couplets
        else:
            return render_template('index.html')    

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e9c6f2ba2bdc.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Feb/2021 14:08:43] "GET /?firstLine=人间 HTTP/1.1" 200 -
